#### Parameter Grid Search

This code does broad grid search for each parameter (or combination if interdependant) seperately. Then, we repeat the grid searh, narrowing in on the minimum error and run time combinations.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import time

from skopt import gp_minimize
from skopt.space import Integer, Real, Categorical
from skopt.utils import use_named_args

from GA_params_class.GeneticAlgorithm import GeneticAlgorithm
from function_module import *

In [2]:
cnn_model_path = '../../Models/CNN_6_1_2.keras'
masked_sequence = 'AATACTAGAGGTCTTCCGACNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGTGTGGGCGGGAAGACAACTAGGGG'
target_expressions = [0, 0.5, 1]
precision = None
verbose = 0
lineages = 10
seed = 0

In [3]:
# Define the parameter ranges for skopt
# Does not include pop_size, islands, gene_flow_rate, num_competitors, boltzmann_temperature
param_ranges = [
    Integer(50, 500, name='generations'),
    Real(0.1, 1.0, name='base_mutation_rate'),
    Integer(1, 30, name='chromosomes'),
    Real(0.1, 1.0, name='elitist_rate'),
    Real(0.1, 1.0, name='surval_rate'),
    Integer(1, 10, name='num_parents'),
    Categorical(['tournament', 'tournament_pop', 'roulette', 
                 'linear_scaling', 'rank_based', 'sus', 'truncation'], name='selection')
]

In [4]:
# Objective function: combines error and runtime
@use_named_args(param_ranges)
def objective_function(**params):
    """
    Objective function for Bayesian Optimization.
    The function combines average_error and runtime.
    Lower weights favor faster runtime optimization.
    """
    try:
        average_error, average_runtime = bayesian_test(
            params,
            cnn_model_path,
            masked_sequence,
            target_expressions,
            precision,
            verbose,
            lineages,
            seed
        )

        return average_error

        # # Combine error and runtime into a single objective
        # weight_error = 0.7  # weight for the error
        # weight_runtime = 0.3  # weight for the runtime
        
        # combined_metric = weight_error * average_error + weight_runtime * average_runtime
        # return combined_metric
    except Exception as e:
        print(e)
        return 1e6 # return a large value in case of an error

In [ ]:
# Perform Bayesian Optimization
result = gp_minimize(
    func=objective_function,
    dimensions=param_ranges,
    n_calls=100,  # Number of evaluations
    random_state=seed,
)

In [ ]:
# Print the results
print("Optimal Parameters:")
print(f"Generations: {result.x[0]}")
print(f"Base Mutation Rate: {result.x[1]}")
print(f"Chromosomes: {result.x[2]}")
print(f"Elitist Rate: {result.x[3]}")
print(f"Survival Rate: {result.x[4]}")
print(f"Num Parents: {result.x[5]}")
print(f"Selection: {result.x[6]}")

print("\nBest Combined Metric:", result.fun)